In [ ]:
import pandas as pd
df = pd.read_excel('Cr-poisoning.xlsx')

In [ ]:
import numpy as np

In [ ]:
def print_confusion_matrix(confusion_matrix, class_names, figsize=(10, 7), fontsize=14):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix.
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

In [ ]:
X1 = df['log pO2 /bar'].tolist()
X2 = df['log CrO3 /bar'].tolist()
X = np.c_[X1,X2]
y = df['reaction product']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
clf = SVC(gamma=0.5, kernel='linear',C = 20)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
from random import sample
idx = sample(list(df.index), int(df.shape[0]))
h = .02
x_min, x_max  = X_train[:, 0].min() - .5, X_train[:, 0].max() + .5
y_min, y_max = X_train[:, 1].min() - .5, X_train[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
encoder = {'SrO': 1, 'SrCrO3': 2, 'Sr3Cr2O8': 3,'Sr2CrO4':4,'SrCrO4':5}
Z = np.array([encoder[z] for z in Z]).reshape(xx.shape)
plt.figure(figsize=(12, 9))
#plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.contourf(xx, yy, Z, cmap=plt.cm.get_cmap('Dark2'), alpha=0.8)
#plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
# Plot also the training points
col_map = {'SrO': 'olive', 'SrCrO3': 'yellow', 'Sr3Cr2O8': 'magenta','Sr2CrO4':"blue",'SrCrO4':"navy"}
plt.scatter(X_0, X_1, c= [col_map[lb] for lb in df['reaction product']], edgecolors='k', cmap=plt.cm.Paired)
#plt.scatter(X_0, X_1, c=y_letransf, s=20, cmap=plt.cm.get_cmap('Dark2'), edgecolors='k')
plt.ylabel('$log_{10}pCrO3_3$')
plt.xlabel('$log_{10}pO_2$')
plt.title("Decision Boundary for SVC")

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.savefig('classification_SVC.png')
plt.show()


In [ ]:
y_hat = clf.predict(X_test)

#y_hat = np.array([item.argmax() for item in y_hat]).reshape(y_test.shape)
print(y_test)
print(y_hat)

confusion_mat = confusion_matrix(y_test, y_hat)

In [ ]:
print_confusion_matrix(confusion_mat, ['Sr2CrO4','Sr3Cr2O8', 'SrCrO3', 'SrCrO4', 'SrO'])
plt.savefig("confusion_matrix_SVC.png")

In [ ]:
len(X_train[:, 0])

In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [ ]:
le = LabelEncoder()
le.fit(y_train)
y_tarin = le.transform(y_train)

In [ ]:
y_test = le.transform(y_test)

In [ ]:
y_tarin = np_utils.to_categorical(y_tarin)

In [ ]:
model = Sequential()
model.add(Dense(5, input_shape = [2,], activation = 'tanh'))
model.add(Dense(5, activation = 'softmax'))

In [ ]:
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,y_tarin,epochs=150,batch_size = 4)

In [ ]:
import matplotlib.pyplot as plt
y_pred = model.predict(X_test)
plt.plot(history.history['acc'])

In [ ]:
y_pred

In [ ]:
y_pred = [item.argmax() for item in y_pred]

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
#X = df.iloc[:, 0:2]
X_0=X[ :, 0]
X_1=X[:,1]
y_letransf = le.fit_transform(y.values)

In [ ]:
import matplotlib.pyplot as plt
from random import sample
idx = sample(list(df.index), int(df.shape[0]))
h = .02
x_min, x_max  = X_train[:, 0].min() - .5, X_train[:, 0].max() + .5
y_min, y_max = X_train[:, 1].min() - .5, X_train[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = np.array([item.argmax() for item in Z]).reshape(xx.shape)
# #Put the result into a color plot
#encoder = {'SrO': 1, 'SrCrO3': 2, 'Sr3Cr2O8': 3,'Sr2CrO4':4,'SrCrO4':5}
#Z = np.array([encoder[z] for z in Z]).reshape(xx.shape)
plt.figure(figsize=(12, 9))
plt.contourf(xx, yy, Z, cmap=plt.cm.get_cmap('Dark2'), alpha=0.8)
#plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
# Plot also the training points
col_map = {'SrO': 'olive', 'SrCrO3': 'yellow', 'Sr3Cr2O8': 'magenta','Sr2CrO4':"blue",'SrCrO4':"navy"}
plt.scatter(X_0, X_1, c= [col_map[lb] for lb in df['reaction product']], edgecolors='k', cmap=plt.cm.Paired)
#plt.scatter(X_0, X_1, c=y_letransf, s=20, cmap=plt.cm.get_cmap('Dark2'), edgecolors='k')
plt.ylabel('$log_{10}pCrO3_3$')
plt.xlabel('$log_{10}pO_2$')
plt.title("Decision Boundary for Neural Network")

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
plt.savefig('classification_NN.png')
plt.show()


In [ ]:
y_hat = model.predict(X_test)

y_hat = np.array([item.argmax() for item in y_hat]).reshape(y_test.shape)
print(y_test)
print(y_hat)

confusion_mat = confusion_matrix(y_test, y_hat)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
print_confusion_matrix(confusion_mat, ['Sr2CrO4','Sr3Cr2O8', 'SrCrO3', 'SrCrO4', 'SrO'])
plt.savefig("confusion_matrix_NN.png")